<h1 style="background-color:#239B56;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
✍ תהליך הסבת שכבת גבולות תכניות לצרכי רישום בשטח לא מוסדר לבנק"ל מודרני ✍ 

In [ ]:
print('4. Unregistered Tazar borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from configs import CNFG
import numpy as np
import pandas as pd
import geopandas as gpd
import subprocess

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('  4.a Reading inputs')

<h1 style="text-align:right;font-size:100%">
שכבת גבולות תצ"ר בשטח לא מוסדר

In [ ]:
unreg_tazar_cols = ['TALAR_NUM', 'TALAR_YEAR', 'SURVEYOR_ID', 'GUSH_NUM', 'GUSH_SUFFIX', 'SURVEY_DATE', 'geometry']

unreg_tazar_layer = gpd.read_file(CNFG.CADSDE + 'TAZAR_BORDER_LOMUSDAR.gdb', layer = 'TAZAR_BORDER_LOMUSDAR', include_fields = unreg_tazar_cols)

<h1 style="text-align:right;font-size:100%">
שכבה מקומית של גבולות תצ"ר ללא מקור, בשטח לא מוסדר

In [ ]:
UnkownUnregiteredazarBorders = gpd.read_file(CNFG.Local + 'UnkownUnregiteredazarBorders.gpkg', layer = 'UnkownUnregiteredazarBorders')

<h1 style="text-align:right;font-size:100%">
שכבת גבולות גושי קדסטר מודרניים

In [ ]:
Blocks = pd.read_csv(CNFG.CSVPath + 'BlockUniqueIDs.csv', usecols = ['Name', 'BlockUniqueID'])

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('  4.b Cleaning data')

In [ ]:
print('  4.c Data conversion')

In [ ]:
unreg_tazar_layer = pd.concat([unreg_tazar_layer, UnkownUnregiteredazarBorders])
unreg_tazar_layer.reset_index(inplace=True)

<h1 style="text-align:right; font-size:100%">
הוספת שדה מקשר של מספר אליפסה מלא לצורך מיזוג

In [ ]:
unreg_tazar_layer.dropna(subset=['TALAR_NUM', 'TALAR_YEAR'], inplace=True)


unreg_tazar_layer[['TALAR_NUM', 'TALAR_YEAR']] = unreg_tazar_layer[['TALAR_NUM', 'TALAR_YEAR']].astype('Int32')


bad_years_cond = unreg_tazar_layer['TALAR_YEAR'] < 1000
unreg_tazar_layer.loc[bad_years_cond, "TALAR_YEAR"] = unreg_tazar_layer['TALAR_YEAR'] + 2000
del bad_years_cond


unreg_tazar_layer['ProcessName'] = unreg_tazar_layer['TALAR_NUM'].astype('str') + '/' + unreg_tazar_layer['TALAR_YEAR'].astype('str')


unreg_tazar_layer.drop(columns=['TALAR_NUM', 'index'], inplace=True)

<h1 style="text-align:right; font-size:100%">
הוספת שדה מקשר של מספר גוש מלא לצורך מיזוג

In [ ]:
unreg_tazar_layer['GUSH_SUFFIX'] = unreg_tazar_layer['GUSH_SUFFIX'].fillna(0)
unreg_tazar_layer[['GUSH_SUFFIX', 'GUSH_NUM']] = unreg_tazar_layer[['GUSH_SUFFIX', 'GUSH_NUM']].astype('Int32')
unreg_tazar_layer['BlockName'] = unreg_tazar_layer['GUSH_NUM'].astype(str) + '/' + unreg_tazar_layer['GUSH_SUFFIX'].astype(str)
unreg_tazar_layer.drop(columns=['GUSH_SUFFIX', 'GUSH_NUM'], inplace=True)

<h1 style="text-align:right; font-size:100%">
השלמת ערכים חסרים בזיהוי מודד

In [ ]:
unreg_tazar_layer['SURVEYOR_ID'] = unreg_tazar_layer['SURVEYOR_ID'].fillna(0)
unreg_tazar_layer['SURVEYOR_ID'] = unreg_tazar_layer['SURVEYOR_ID'].astype('Int32')

<h1 style="text-align:right; font-size:100%">
השלמת ערכים חסרים בתאריך המדידה והפקת שנת המדידה

In [ ]:
unreg_tazar_layer['SURVEY_DATE'] = pd.to_datetime(unreg_tazar_layer['SURVEY_DATE'])
unreg_tazar_layer['SurveyYear'] = unreg_tazar_layer['SURVEY_DATE'].dt.year
unreg_tazar_layer['SurveyYear'] = unreg_tazar_layer['SurveyYear'].fillna(unreg_tazar_layer['TALAR_YEAR'])
unreg_tazar_layer['SurveyYear'] = unreg_tazar_layer['SurveyYear'] .astype('Int32')


unreg_tazar_layer.drop(columns=['SURVEY_DATE', 'TALAR_YEAR'], inplace=True)

<h1 style="text-align:right; font-size:100%">
הסרת רשומות כפולות לפי מספר אליפסה וגיאומטריה זהים

In [ ]:
unreg_tazar_layer = unreg_tazar_layer[~unreg_tazar_layer.duplicated(subset=['ProcessName', 'geometry'], keep = 'first')].sort_values('ProcessName')
unreg_tazar_layer = unreg_tazar_layer.drop_duplicates(subset=['ProcessName', 'SURVEYOR_ID'], keep='first')

<h1 style="text-align:right; font-size:100%">
איחוד עם טבלת מזהי גושים לקבלת מספר מזהה של הגוש

In [ ]:
unreg_tazar_layer = unreg_tazar_layer.merge(Blocks, left_on = 'BlockName', right_on = 'Name', how = 'left')
unreg_tazar_layer.drop(columns = 'Name', inplace = True)

del Blocks

<h1 style="text-align:right; font-size:100%">
יצירה והגדרת שדה של מקור הנתונים

In [ ]:
CAD_cond = unreg_tazar_layer['SurveyYear'] <= 2015
unreg_tazar_layer['DataSource'] = np.where(CAD_cond, 5, 4)  # CAD source:5, SRV source:4

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [ ]:
print('  4.d Defining new Layer')

In [ ]:
new_columns = np.dtype([
                        ("CPBUniqueID", int),
                        ("BlockUniqueID", int),
                        ("ProcessType", int),
                        ("MapYear", int),
                        ("ProcessName", str),
                        ("GeodeticNetwork", int),
                        #("Status", int),
                        #("PlanName", str),
                        ("SurveyorLicenseID", int),
                        ("DataSource", int),
                        ("geometry", gpd.array.GeometryDtype)
                        ])

unreg_tazar_layer_new = gpd.GeoDataFrame(np.empty(0, dtype=new_columns),
                                         geometry = 'geometry',
                                         crs = 2039)

del new_columns

<h1 style="text-align:right; font-size:100%">
איכלוס שדות

In [ ]:
unreg_tazar_layer_new['geometry'] = unreg_tazar_layer['geometry']


unreg_tazar_layer_new['CPBUniqueID'] = range(1, len(unreg_tazar_layer)+1)
unreg_tazar_layer_new['CPBUniqueID'] = unreg_tazar_layer_new['CPBUniqueID'] + 11000000
unreg_tazar_layer_new['CPBUniqueID'] = unreg_tazar_layer_new['CPBUniqueID'].astype('Int64')

unreg_tazar_layer_new['MapYear'] = unreg_tazar_layer['SurveyYear'].astype('Int64')


unreg_tazar_layer_new['ProcessName'] = unreg_tazar_layer['ProcessName'].astype('str')


unreg_tazar_layer_new['ProcessType'] = 11  #תצ"ר בשטח לא מוסדר



unreg_tazar_layer_new['DataSource'] = unreg_tazar_layer['DataSource'].astype('Int64')


#unreg_tazar_layer_new['Status'] = 0  unknown, will be Null in product gdb


geodeticNetDict = {range(0, 1998)    : 1,      # רשת ישראל הישנה
                   range(1999, 2007) : 2,      # רשת ישראל החדשה
                   range(2008, 99999): 3}      # רשת ישראל התקפה
unreg_tazar_layer_new['GeodeticNetwork'] = unreg_tazar_layer['SurveyYear'].apply(lambda x: next((v for k, v in geodeticNetDict.items() if x in k), 0))
unreg_tazar_layer_new['GeodeticNetwork'] = unreg_tazar_layer_new['GeodeticNetwork'].astype('Int64')


unreg_tazar_layer_new['SurveyorLicenseID'] = unreg_tazar_layer['SURVEYOR_ID'].fillna(0)
unreg_tazar_layer_new['SurveyorLicenseID'] = unreg_tazar_layer_new['SurveyorLicenseID'].astype('Int64')

#unreg_tazar_layer_new['PlanName'] = None

unreg_tazar_layer_new['BlockUniqueID'] = unreg_tazar_layer['BlockUniqueID'].fillna(0)
unreg_tazar_layer_new['BlockUniqueID'] = unreg_tazar_layer_new['BlockUniqueID'].astype('Int64')

del unreg_tazar_layer

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כטבלה
<h1 style="text-align:right; font-size:100%;"> 
שמירה כשכבה גיאוגרפית

In [ ]:
print('  4.e Exporting results to product folder')

In [ ]:
json_dir =  CNFG.JsonPath +'UnregisteredTazar.geojson'
unreg_tazar_layer_new.to_file(json_dir, driver = 'GeoJSON')

In [ ]:
subprocess.call([CNFG.BatchPath + 'UnregisteredTazar-Save as Feature Class.bat']);